In [ ]:
import numpy as np

# fmt: off
input = [3,8,1001,8,10,8,105,1,0,0,21,38,63,80,105,118,199,280,361,442,99999,3,9,102,5,9,9,1001,9,3,9,1002,9,2,9,4,9,99,3,9,1001,9,4,9,102,4,9,9,101,4,9,9,102,2,9,9,101,2,9,9,4,9,99,3,9,1001,9,5,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,1002,9,5,9,101,3,9,9,102,5,9,9,101,3,9,9,4,9,99,3,9,1002,9,2,9,1001,9,4,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,99,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,99]
# fmt: on

In [ ]:
# ABCDE
# DE - two-digit opcode,      02 == opcode 2
# C - mode of 1st parameter,  0 == position mode
# B - mode of 2nd parameter,  1 == immediate mode
# A - mode of 3rd parameter,  0 == position mode,


def decode(c):  # returns mode_a,mode_b,mode_c,opcode
    code_5 = "{:05d}".format(c)
    return (
        int(code_5[0]),
        int(code_5[1]),
        int(code_5[2]),
        10 * int(code_5[3]) + int(code_5[4]),
    )


def function(code, input_signals=None):
    i = 0
    input_index = 0

    mode_a, mode_b, mode_c, opcode = decode(code[i])

    while opcode != 99:

        # print(code[i],mode_a,mode_b,mode_c,opcode)

        if mode_c == 0:
            i1 = code[i + 1]
        else:
            i1 = i + 1
        if mode_b == 0:
            i2 = code[i + 2]
        else:
            i2 = i + 2
        if mode_a == 0 and i + 3 < len(code):
            i3 = code[i + 3]
        else:
            i3 = i + 3

        if opcode == 1:  # addition
            code[i3] = code[i1] + code[i2]
            i = i + 4
        elif opcode == 2:  # multiplication
            code[i3] = code[i1] * code[i2]
            i = i + 4
        elif opcode == 3:  # input
            code[i1] = input_signals[input_index]
            input_index += 1
            i = i + 2
        elif opcode == 4:  # output
            OUTPUT = code[i1]
            # print(OUTPUT)
            i = i + 2
        elif opcode == 5:  # jump-if-true
            if code[i1] != 0:
                i = code[i2]
            else:
                i = i + 3
        elif opcode == 6:  # jump-if-false
            if code[i1] == 0:
                i = code[i2]
            else:
                i = i + 3
        elif opcode == 7:  # less than
            if code[i1] < code[i2]:
                code[i3] = 1
            else:
                code[i3] = 0
            i = i + 4
        elif opcode == 8:  # equals
            if code[i1] == code[i2]:
                code[i3] = 1
            else:
                code[i3] = 0
            i = i + 4

        mode_a, mode_b, mode_c, opcode = decode(code[i])

    # print("HALT")
    return OUTPUT

In [ ]:
import itertools


def find_best_sequence(program):
    # 01234
    max_thrust = 0
    best_seq = None
    perms = list(itertools.permutations([0, 1, 2, 3, 4]))

    for pp in perms:
        # print(pp)
        input_signal = 0
        output_signal = 0
        for prog in range(0, 5):  # a b c d e
            # start program
            input_signals = [pp[prog], input_signal]
            # print(input_signals)
            output_signal = function(program.copy(), input_signals)
            input_signal = output_signal
        if output_signal > max_thrust:
            max_thrust = output_signal
            best_seq = pp
    return best_seq, max_thrust

In [ ]:
def test(cc):
    print(str(cc) + " -> " + str(find_best_sequence(cc)))

In [ ]:
# test() #

program = [3, 15, 3, 16, 1002, 16, 10, 16, 1, 16, 15, 15, 4, 15, 99, 0, 0]
test(program)  # 43210 (from phase setting sequence 4,3,2,1,0):

program = [
    3,
    23,
    3,
    24,
    1002,
    24,
    10,
    24,
    1002,
    23,
    -1,
    23,
    101,
    5,
    23,
    23,
    1,
    24,
    23,
    23,
    4,
    23,
    99,
    0,
    0,
]
test(program)  # 54321 (from phase setting sequence 0,1,2,3,4):

program = [
    3,
    31,
    3,
    32,
    1002,
    32,
    10,
    32,
    1001,
    31,
    -2,
    31,
    1007,
    31,
    0,
    33,
    1002,
    33,
    7,
    33,
    1,
    33,
    31,
    31,
    1,
    32,
    31,
    31,
    4,
    31,
    99,
    0,
    0,
    0,
]
test(program)  # 65210 (from phase setting sequence 1,0,4,3,2):

In [ ]:
find_best_sequence(input)